In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Импорт библиотек и создание папок для сохранения данных

In [30]:
import os
import re
import csv
import requests
import urllib.request
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time

DIRECTORY='/content/drive/MyDrive/Houses/'
PATH_1='city_1/' #Нижний Новгород
PATH_2='city_2/' #Казань
PATH_3='city_3/' #Новосибирск

if not os.path.isdir(DIRECTORY):
     os.mkdir(DIRECTORY)

if not os.path.isdir(DIRECTORY+PATH_1):
     os.mkdir(DIRECTORY+PATH_1)

if not os.path.isdir(DIRECTORY+PATH_2):
     os.mkdir(DIRECTORY+PATH_2)

if not os.path.isdir(DIRECTORY+PATH_3):
     os.mkdir(DIRECTORY+PATH_3)

## Создание функции для получении характеристик дома из ссылки

In [3]:
# Запрос на получение данных со страницы 
url = 'https://dom.mingkh.ru/nizhegorodskaya-oblast/nizhniy-novgorod/461518'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
#print(soup)
#print(r)

In [4]:
# Выбор параметров из полученных данных
house_properties = soup.find(class_='dl-horizontal house')
house_info = soup.find_all(class_='table-responsive')
params_list_1 = [x.text for x in house_properties.find_all('dt')]
params_list_2 = [x.text for x in house_info[0].find_all('td')] \
    + [x.text for x in house_info[1].find_all('td')] \
    + [x.text for x in house_info[2].find_all('td')]

# Важная строка для последующего парсинга
#house_properties.find_all('dt')
#house_properties.find_all('dd')

In [5]:
# Смотрим параметры в переменной params_list_1
params_list_1

['Адрес',
 'Год постройки',
 'Количество этажей',
 'Тип дома',
 'Жилых помещений',
 'Капитальный ремонт',
 'Серия, тип постройки',
 'Тип перекрытий',
 'Материал несущих стен',
 'Тип мусоропровода',
 'Дом признан аварийным',
 'Выписка из ЕГРН\nКак получить выписку?\n\xa0\n\xa0\nДетская площадка\nНе имеется\nСпортивная площадка\nНе имеется\nКадастровый номер\n52:18:0040416:39\nУправляющая компания\nТСЖ «ТСЖ №153»\r\n                                            с 28.03.2006                                    \n\xa0\n\xa0Оставить\r\n                    отзыв о доме или УК\r\n            \n',
 '\nКак получить выписку?\n\xa0\n\xa0\nДетская площадка\nНе имеется\nСпортивная площадка\nНе имеется\nКадастровый номер\n52:18:0040416:39\nУправляющая компания\nТСЖ «ТСЖ №153»\r\n                                            с 28.03.2006                                    \n\xa0\n\xa0Оставить\r\n                    отзыв о доме или УК\r\n            \n',
 '\xa0',
 'Детская площадка',
 'Спортивная площадка

In [6]:
# Смотрим параметры в переменной params_list_2
params_list_2

['Год ввода в эксплуатацию',
 '1970',
 'Детская площадка',
 'Не имеется',
 'Дом признан аварийным',
 'Нет',
 'Дополнительная информация',
 'не имеется',
 'Количество жилых помещений',
 '60',
 'Класс энергетической эффективности',
 'Не присвоен',
 'Количество подъездов',
 '4',
 'Наибольшее количество этажей',
 '5',
 'Наименьшее количество этажей',
 '5',
 'Площадь жилых помещений м2',
 '2 129.00',
 'Площадь нежилых помещений м2',
 '915.00',
 'Площадь помещений общего имущества м2',
 '915.00',
 'Площадь зем. участка общего имущества м2',
 '3 908.00',
 'Площадь парковки м2',
 '0.00',
 'Серия, тип постройки здания',
 '1464А-77',
 'Спортивная площадка',
 'Не имеется',
 'Формирование фонда кап. ремонта',
 'На специальном счете организации',
 'Тип дома',
 'Многоквартирный дом',
 'Количество вводов в дом, ед.\n1\n',
 '\n1\n',
 '1',
 'Объем выгребных ям, куб. м.\n0.00\n',
 '\n0.00\n',
 '0.00',
 'Вентиляция\nПриточно-вытяжная вентиляция\n',
 '\nПриточно-вытяжная вентиляция\n',
 'Приточно-вытяжная

In [31]:
# Функция для получении свойств из ссылки конкретного адреса дома:
def house_params(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    house_properties = soup.find(class_='dl-horizontal house')
    house_info = soup.find_all(class_='table-responsive')
    
    def get_param_block_1(param):
        try:
            params_list_1 = [x.text for x in house_properties.find_all('dt')]
            param_out = house_properties.find_all('dd')[params_list_1.index(param)].text.lower()
        except:
            param_out = 'отсутствует'
        return param_out
    
    def get_param_block_2(param):
        try:
            params_list_2 = [x.text for x in house_info[0].find_all('td')] \
                + [x.text for x in house_info[1].find_all('td')]
            param_out = params_list_2[params_list_2.index(param) + 1].lower()
        except:
            param_out = 'не известно'
        return param_out
    
    if (house_properties != None) and (house_info != None):
        adress = get_param_block_1('Адрес').split('\xa0')[0]
        building_year = get_param_block_1('Год постройки')
        number_of_floors = get_param_block_1('Количество этажей')
        type_house = get_param_block_1('Тип дома')
        #number_of_flats = get_param_block_1('Жилых помещений')
        series_of_house = get_param_block_1('Серия, тип постройки')
        type_flooring = get_param_block_1('Тип перекрытий')
        carry_wall_mater = get_param_block_1('Материал несущих стен')
        is_garbage_chute = get_param_block_1('Тип мусоропровода')
        is_house_emergency = get_param_block_1('Дом признан аварийным')
        year_of_population = get_param_block_2('Год ввода в эксплуатацию')
        is_child_area = get_param_block_2('Детская площадка')
        is_sport_area = get_param_block_2('Спортивная площадка')
        #cadastral_number = get_param_block_2('Кадастровый номер')
        #manage_company = get_param_block_2('Управляющая компания')
        year_commissioning = get_param_block_2('Год ввода в эксплуатацию')
        number_living_rooms = get_param_block_2('Количество жилых помещений')
        energy_safety_class = get_param_block_2('Класс энергетической эффективности')
        number_of_lifts = get_param_block_2('Количество лифтов')
        number_of_entrances = get_param_block_2('Количество подъездов')
        lifts_types = get_param_block_2('Лифты')
        max_num_floors = get_param_block_2('Наибольшее количество этажей')
        min_num_floors = get_param_block_2('Наименьшее количество этажей')
        number_non_living_rooms = get_param_block_2('Количество нежилых помещений')
        area_living_rooms = get_param_block_2('Площадь жилых помещений м2')
        area_non_living_rooms = get_param_block_2('Площадь нежилых помещений м2')
        area_collective_rooms = get_param_block_2('Площадь помещений общего имущества м2')
        area_house_land = get_param_block_2('Площадь зем. участка общего имущества м2')
        area_parking = get_param_block_2('Площадь парковки м2')
        
        return [adress, building_year, number_of_floors, type_house, series_of_house, 
                type_flooring, carry_wall_mater, is_garbage_chute, is_house_emergency, 
                year_of_population, is_child_area, is_sport_area, #cadastral_number, manage_company, 
                year_commissioning, number_living_rooms, energy_safety_class,
                number_of_lifts, number_of_entrances, lifts_types, max_num_floors, 
                min_num_floors, number_non_living_rooms, area_living_rooms, area_non_living_rooms,
                area_collective_rooms, area_house_land, area_parking]

In [10]:
# проверка 1 работы функции
#url = 'https://dom.mingkh.ru/nizhegorodskaya-oblast/nizhniy-novgorod/461518'
#house_params(url)

# проверка 2 работы функции
#url = 'https://dom.mingkh.ru/nizhegorodskaya-oblast/nizhniy-novgorod/828677'
#house_params(url)

['ул. дубравная 10-я, д. 2, нижний новгород, нижегородская область',
 '1958',
 '2',
 'многоквартирный дом',
 '-',
 'деревянные',
 'блочные',
 'отсутствует',
 'нет',
 '1958',
 'не имеется',
 'не имеется',
 '1958',
 'не известно',
 'не известно',
 'не известно',
 '1',
 'не известно',
 '2',
 'не известно',
 'не известно',
 '325.10',
 '0.00',
 'не известно',
 '917.00',
 '0.00']

## Получения всех ссылок по городу

In [ ]:
# Парсинг ссылок всех домов по городу Нижний Новгород
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 
list_of_links = []

pattern_1 = re.compile(r'href="(.*)">')
i=1

part_1_url = 'https://dom.mingkh.ru/nizhegorodskaya-oblast/nizhniy-novgorod/'
part_2_url = 'houses?page='
url = part_1_url + part_2_url

request=urllib.request.Request(url+str(i),None,headers)
resp = urllib.request.urlopen(request)
soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))

#определение последней страницы
last_page = [int(re.search('\d+', str(link))[0]) for link in soup.find_all('a', href=True) \
    if len(re.findall('Последняя', str(link))) != 0][0]
print(f'Total number of pages with links: {last_page}')

# цикл по страницам + сбор всех ссылок в файл
for page in range(1, last_page+1):
    request=urllib.request.Request(url+str(page), None, headers)
    resp = urllib.request.urlopen(request)
    soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))
    for link in soup.find_all('a', href=True):
        if (re.search('\d{4}', str(link)) != None):
            list_of_links.append('https://dom.mingkh.ru'+re.search(pattern_1, str(link))[0][6:-2])
    if page%10==0:
        print(f'{page} processed out of {last_page}')

print(len(list_of_links))

# запись всего списка в файл txt
with open(DIRECTORY+PATH_1+"houses_links_full_1.txt", "w") as file:
    print(*list_of_links, file=file, sep="\n")

Total number of pages with links: 107
10 processed out of 107
20 processed out of 107
30 processed out of 107
40 processed out of 107
50 processed out of 107
60 processed out of 107
70 processed out of 107
80 processed out of 107
90 processed out of 107
100 processed out of 107
10659


In [ ]:
# Парсинг ссылок всех домов по городу Казань
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 
list_of_links = []

pattern_1 = re.compile(r'href="(.*)">')
i=1

part_1_url = 'https://dom.mingkh.ru/tatarstan/kazan/'
part_2_url = 'houses?page='
url = part_1_url + part_2_url

request=urllib.request.Request(url+str(i),None,headers)
resp = urllib.request.urlopen(request)
soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))

# определение последней страницы
last_page = [int(re.search('\d+', str(link))[0]) for link in soup.find_all('a', href=True) \
    if len(re.findall('Последняя', str(link))) != 0][0]
print(f'Total number of pages with links: {last_page}')

# цикл по страницам + сбор всех ссылок в файл
for page in range(1, last_page+1):
    request=urllib.request.Request(url+str(page), None, headers)
    resp = urllib.request.urlopen(request)
    soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))
    for link in soup.find_all('a', href=True):
        if (re.search('\d{3}', str(link)) != None):
            list_of_links.append('https://dom.mingkh.ru'+re.search(pattern_1, str(link))[0][6:-2])
    if page%10==0:
        print(f'{page} processed out of {last_page}')

print(len(list_of_links))

# запись всего списка в файл txt
with open(DIRECTORY+PATH_2+"houses_links_full_2.txt", "w") as file:
    print(*list_of_links, file=file, sep="\n")

Total number of pages with links: 63
10 processed out of 63
20 processed out of 63
30 processed out of 63
40 processed out of 63
50 processed out of 63
60 processed out of 63
6176


## Парсинг характеристик домов при проходе по полученным ссылкам (см. выше)

In [ ]:
# Запись данных парсинга в файл csv по городу 1 (Нижний Новгород)
total_table = []

# Если сервер обрывает связь, посмотреть в записываемом файле 
# на какой строке остановилась запись и добавить/раскомментить skp_rows

#skp_rows = 7295

with open(DIRECTORY+PATH_1+"houses_links_full_1.txt", 'r') as f:
    #for _ in range(1, skp_rows+1):
    #    next(f)
    for k, line in enumerate(f, start=1): #start=skp_rows+1
        if k%100==0: # печатаем links proceed каждые 100 ссылок
            print(f'{k} links proceed') 
            time.sleep(3)
        if k%300==0: # каждые 300 ссылок делаем паузу на 15 сек
            time.sleep(15)
        url = line.strip()
        total_table.append(house_params(url))
        
        if k%10==0: # каждые 10 ссылок пишем данные в файл
            with open(DIRECTORY+PATH_1+'houses_1.csv', "a") as file: 
                writer = csv.writer(file, delimiter =';') 
                writer.writerows(total_table)
            total_table = []
print(total_table)
# дописываем данные по оставшимся ссылкам (меньше 10 шт в конце)
with open(DIRECTORY+PATH_1+'houses_1.csv', "a") as file: 
    writer = csv.writer(file, delimiter =';')
    writer.writerows(total_table)

7300 links proceed
7400 links proceed
7500 links proceed
7600 links proceed
7700 links proceed
7800 links proceed
7900 links proceed
8000 links proceed
8100 links proceed
8200 links proceed
8300 links proceed
8400 links proceed
8500 links proceed
8600 links proceed
8700 links proceed
8800 links proceed
8900 links proceed
9000 links proceed
9100 links proceed
9200 links proceed
9300 links proceed
9400 links proceed
9500 links proceed
9600 links proceed
9700 links proceed
9800 links proceed
9900 links proceed
10000 links proceed
10100 links proceed
10200 links proceed
10300 links proceed
10400 links proceed
10500 links proceed
10600 links proceed
[['кп. зеленый город, п. д/о красное сормово, д. 5, нижний новгород, нижегородская область', '1971', '2', 'многоквартирный дом', 'панельный', 'смешанные', 'смешанные', 'отсутствует', 'нет', '1971', 'не имеется', 'не имеется', '1971', '7', 'не присвоен', '1', '1', 'не известно', '2', '2', 'не известно', '285.80', '0.00', '0.00', '0.00', '0.00'], 

In [ ]:
# Запись данных парсинга в файл csv по городу 2 (Казань)
total_table = []

# Если сервер обрывает связь, посмотреть в записываемом файле 
# на какой строке остановилась запись и добавить/раскомментить skp_rows

#skp_rows = 2491
with open (DIRECTORY+PATH_2+"houses_links_full_2.txt", 'r') as f:
    #for _ in range(1, skp_rows+1):
    #    next(f)
    for k, line in enumerate(f, start=1): #start=skp_rows+1
        if k%100==0: # печатаем links proceed каждые 100 ссылок
            print(f'{k} links proceed')
            time.sleep(3)
        if k%300==0: # каждые 300 ссылок делаем паузу на 15 сек
            time.sleep(15)
        url = line.strip()
        total_table.append(house_params(url))
        if k%10==0: # каждые 10 ссылок пишем данные в файл
            with open(DIRECTORY+PATH_2+'houses_2.csv', "a") as file: 
                writer = csv.writer(file, delimiter =';') 
                writer.writerows(total_table)
            total_table = []
#print(total_table)

# дописываем данные по оставшимся ссылкам (меньше 10 шт в конце)
with open(DIRECTORY+PATH_2+'houses_2.csv', "a") as file: 
    writer = csv.writer(file, delimiter =';')
    writer.writerows(total_table)

2500 links proceed
2600 links proceed
2700 links proceed
2800 links proceed
2900 links proceed
3000 links proceed
3100 links proceed
3200 links proceed
3300 links proceed
3400 links proceed
3500 links proceed
3600 links proceed
3700 links proceed
3800 links proceed
3900 links proceed
4000 links proceed
4100 links proceed
4200 links proceed
4300 links proceed
4400 links proceed
4500 links proceed
4600 links proceed
4700 links proceed
4800 links proceed
4900 links proceed
5000 links proceed
5100 links proceed
5200 links proceed
5300 links proceed
5400 links proceed
5500 links proceed
5600 links proceed
5700 links proceed
5800 links proceed
5900 links proceed
6000 links proceed
6100 links proceed
[['м. ферма 2, д. 98б, казань, татарстан', '2015', '10', 'многоквартирный дом', 'индивид.', 'железобетонные', 'иные', 'на лестничной клетке', 'нет', '2015', 'не имеется', 'не имеется', '2015', '66', 'b', '1', '1', 'пассажирские - 1, ', '10', '9', '2', '1 879.30', '2 277.70', '1 433.80', '2 267.00

In [13]:
columns_list =  ['adress', 'building_year', 'number_of_floors', 'type_house', 'series_of_house', 
                 'type_flooring', 'carry_wall_mater', 'is_garbage_chute', 'is_house_emergency', 
                 'year_of_population', 'is_child_area', 'is_sport_area', 'year_commissioning', 
                 'number_living_rooms', 'energy_safety_class', 'number_of_lifts', 
                 'number_of_entrances', 'lifts_types', 'max_num_floors', 'min_num_floors', 
                 'number_non_living_rooms', 'area_living_rooms', 'area_non_living_rooms', 
                 'area_collective_rooms', 'area_house_land', 'area_parking']
df_1 = pd.read_csv(DIRECTORY+PATH_1+'houses_1.csv', delimiter=';', names=columns_list)
df_2 = pd.read_csv(DIRECTORY+PATH_2+'houses_2.csv', delimiter=';', names=columns_list)

In [14]:
df_1.head()

,adress,building_year,number_of_floors,type_house,series_of_house,type_flooring,carry_wall_mater,is_garbage_chute,is_house_emergency,year_of_population,...,number_of_entrances,lifts_types,max_num_floors,min_num_floors,number_non_living_rooms,area_living_rooms,area_non_living_rooms,area_collective_rooms,area_house_land,area_parking
0,"пер. 1-й кемеровский, д. 2/12, нижний новгород...",1952,2,многоквартирный дом,отсутствует,деревянные,блочные,отсутствует,нет,1952,...,1,не известно,2,2,не известно,542.30,163.00,657.40,1 801.00,не известно
1,"пер. 1-й кемеровский, д. 3, нижний новгород, н...",1958,2,многоквартирный дом,отсутствует,деревянные,кирпич,отсутствует,нет,1958,...,1,не известно,2,2,не известно,435.00,0.00,332.80,1 236.00,не известно
2,"пер. 1-й кемеровский, д. 4, нижний новгород, н...",1952,2,многоквартирный дом,отсутствует,деревянные,иные,отсутствует,нет,1952,...,1,не известно,2,2,не известно,680.00,49.70,672.80,2 002.00,не известно
3,"пер. 1-й кемеровский, д. 5, нижний новгород, н...",1960,2,многоквартирный дом,отсутствует,деревянные,кирпич,отсутствует,нет,1960,...,1,не известно,2,2,не известно,421.90,0.00,327.20,1 801.00,не известно
4,"пер. 1-й кемеровский, д. 6, нижний новгород, н...",1955,2,многоквартирный дом,отсутствует,деревянные,иные,отсутствует,нет,1955,...,1,не известно,2,2,не известно,679.40,175.10,916.20,2 510.00,не известно


In [ ]:
print('Число домов в Нижнем Новогороде', len(df_1))
print('Число домов в Казани', len(df_2))

Число домов в Нижнем Новогороде 10659
Число домов в Казани 6176


In [ ]:
df_1.series_of_house.unique()

array(['отсутствует', 'народная стройка', '-', 'нет', 'кирпичный',
       'деревянный', 'бревенчатый', 'блочный', '1-464 д-1',
       'деревянный времён первых пятилеток', '1-464д-86', '1-464а-77',
       '1464а-77', '1-464f-77', '1-464-77', '1-464а', '1-4642-86',
       '1-464а-74', 'индивидуальный', '1', 'панельный', '1-464д-89',
       '1-464д-84', 'э-93', '03-12-5а-ар', 'панельные', '1-464-а-74',
       'i-464а-76', '1-464а-75 с подвалом', 'тп1-464а-76', 'тп 1-464 н-1',
       '1-464д-83', '51-07', '04-12-5б-ар', '02-12-4б-ар', '01-12-4а-ар',
       '089-003-09', 'и-155н', '021/16-дстр-14-ар', 'маневренный фонд',
       'нет данных', '461д', '1-447с-38', 'серия 1-464а (1-10)',
       '1-464а (1-10)', '1-464а-1', '1-464а-3', '1-464а -1', 'индивид.',
       '1-447с', '1-464а-5', '1-447с-5', 'брусчатый', 'тп1-464д-89',
       '1-464д типовой проект 1-464-85', '1-466', 'тп1-464-д-83',
       '1-464-а-77', '166-22-13-19', '166-10/13-16', '165-21-11-5-ар',
       '166-22-11-2-п3', '166-2

In [17]:
v_cnts_1 = df_1.series_of_house.value_counts()
v_cnts_1.loc[v_cnts_1 > 10]

отсутствует                           3825
кирпичный                             1192
-                                      841
нет                                    521
нет данных                             506
нет данных.                            432
панельный                              268
дореволюционный                        239
индивидуальный                         224
народная стройка                       183
блочный                                149
деревянный времён первых пятилеток     129
индивид                                120
1-466                                   94
1-447с                                  67
461д                                    61
общежитие                               48
брусчатый                               45
панельный 1-466                         45
бревенчатый                             38
1-464а-1                                36
1-464д/1                                35
индивидуальный проект                   34
типовой ооо

In [21]:
# приближенная оценка числа данных по сериям домов, которым данные отсуствовали
v_cnts_1.loc[v_cnts_1 > 100].sum()/len(df_1)

0.8095506145041749

In [22]:
df_2.series_of_house.unique()

array(['индивид.', 'отсутствует', '1-467 а-15', '1-447 с-40', '1-511',
       '124', '1-464 а', '86-014', '1-305', '1-447 с-43', '1-447 с-51',
       'неопределен', '86', '1-447 с-41', '85', '1р-447 с-54', '515',
       '1р-447 с-25/65', '1-464 а-1', '125', '121', '164-80-64', '1-447',
       '1-464 д-105', '1-467 а', '468с', '1 - 528- кп80 эк', 'монолит',
       '2-04-65-1', '111-90', 'с-121', '73-05', '1-510', '1-467 д-17',
       '1-447 с-54/73', '90ло-м', '11-08', '11-01', '11-49-03', 'и-522а',
       '11-66', '114-86-28', 'ii-07', '81', '1-468 а-7', '25', '11-34',
       '164-80-4', '1-447 с-5', '1-447-с-5/60', 'и-491а', '1-515-1819',
       '1-468 б-1, 2', '1-464 д-104', 'c-84', '86-101-44', '1-468 р-1,2',
       '83', '160-80-4', 'э-0907 (141)', '114-85-2', '1-240 с-7',
       '1605/5', '1-467 д-15', '86-034.85', '1-467 д-18', '114-86-1',
       '1-528кп-40', 'тп 164-80-4', '114-86-1 с-16', '114-86',
       '1-468 р-3, 4', '1-467 а-18', 'э-0931/1.5', '-', '1-447 с',
       '1-44

In [23]:
df_2.series_of_house.value_counts()

индивид.       1385
отсутствует     822
1-447 с-40      714
1-511           381
468с            262
               ... 
э-0931/1.5        1
с-83              1
86-034.85         1
606               1
464               1
Name: series_of_house, Length: 119, dtype: int64

In [27]:
v_cnts_2 = df_2.series_of_house.value_counts()
v_cnts_2.loc[v_cnts_2 > 10]

индивид.            1385
отсутствует          822
1-447 с-40           714
1-511                381
468с                 262
неопределен          193
ii-07                190
121                  185
86                   134
1-464 а              109
125                  106
1-447 с-41           105
11-01                 95
111-90                93
1-447 с-54/73         70
1-447 с-51            66
1-447                 53
1-447 с-43            52
1р-447 с-54           51
90ло-м                50
1р-447 с-25/65        48
164-80-64             48
1-467 а               46
124                   44
447 c                 41
11-49-03              36
73-05                 32
1-464 а-1             31
1-515-1819            31
83                    31
монолит               29
11-34                 25
2-04-65-1             25
11-66                 24
1605/5                24
1-467 а-15            23
1-447 с-38            22
515                   21
1-447 с-34            21
1 - 528- кп80 эк      21


In [40]:
# приближенная оценка числа данных по сериям домов, которым данные отсуствовали
(v_cnts_2.loc[v_cnts_2 > 800].sum()+193)/len(df_2)

0.38860103626943004

## Дополнительный анализ города Новосибирск

In [32]:
# Парсинг ссылок всех домов по городу Новосибирск
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 
list_of_links = []

pattern_1 = re.compile(r'href="(.*)">')
i=1

part_1_url = 'https://dom.mingkh.ru/novosibirskaya-oblast/novosibirsk/'
part_2_url = 'houses?page='
url = part_1_url + part_2_url

request=urllib.request.Request(url+str(i),None,headers)
resp = urllib.request.urlopen(request)
soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))

#определение последней страницы
last_page = [int(re.search('\d+', str(link))[0]) for link in soup.find_all('a', href=True) \
    if len(re.findall('Последняя', str(link))) != 0][0]
print(f'Total number of pages with links: {last_page}')

# цикл по страницам + сбор всех ссылок в файл
for page in range(1, last_page+1):
    request=urllib.request.Request(url+str(page), None, headers)
    resp = urllib.request.urlopen(request)
    soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))
    for link in soup.find_all('a', href=True):
        if (re.search('\d{4}', str(link)) != None):
            list_of_links.append('https://dom.mingkh.ru'+re.search(pattern_1, str(link))[0][6:-2])
    if page%10==0:
        print(f'{page} processed out of {last_page}')

print(len(list_of_links))

# запись всего списка в файл txt
with open(DIRECTORY+PATH_3+"houses_links_full_3.txt", "w") as file:
    print(*list_of_links, file=file, sep="\n")

Total number of pages with links: 86
10 processed out of 86
20 processed out of 86
30 processed out of 86
40 processed out of 86
50 processed out of 86
60 processed out of 86
70 processed out of 86
80 processed out of 86
8541


In [33]:
# Запись данных парсинга в файл csv по городу 3 (Новосибирск)
total_table = []

# Если сервер обрывает связь, посмотреть в записываемом файле 
# на какой строке остановилась запись и добавить/раскомментить skp_rows

#skp_rows = 2491
with open (DIRECTORY+PATH_3+"houses_links_full_3.txt", 'r') as f:
    #for _ in range(1, skp_rows+1):
    #    next(f)
    for k, line in enumerate(f, start=1): #start=skp_rows+1
        if k%100==0: # печатаем links proceed каждые 100 ссылок
            print(f'{k} links proceed')
            time.sleep(5)
        if k%300==0: # каждые 300 ссылок делаем паузу на 15 сек
            time.sleep(20)
        url = line.strip()
        total_table.append(house_params(url))
        if k%10==0: # каждые 10 ссылок пишем данные в файл
            with open(DIRECTORY+PATH_3+'houses_3.csv', "a") as file: 
                writer = csv.writer(file, delimiter =';') 
                writer.writerows(total_table)
            total_table = []
#print(total_table)

# дописываем данные по оставшимся ссылкам (меньше 10 шт в конце)
with open(DIRECTORY+PATH_3+'houses_3.csv', "a") as file: 
    writer = csv.writer(file, delimiter =';')
    writer.writerows(total_table)

100 links proceed
200 links proceed
300 links proceed
400 links proceed
500 links proceed
600 links proceed
700 links proceed
800 links proceed
900 links proceed
1000 links proceed
1100 links proceed
1200 links proceed
1300 links proceed
1400 links proceed
1500 links proceed
1600 links proceed
1700 links proceed
1800 links proceed
1900 links proceed
2000 links proceed
2100 links proceed
2200 links proceed
2300 links proceed
2400 links proceed
2500 links proceed
2600 links proceed
2700 links proceed
2800 links proceed
2900 links proceed
3000 links proceed
3100 links proceed
3200 links proceed
3300 links proceed
3400 links proceed
3500 links proceed
3600 links proceed
3700 links proceed
3800 links proceed
3900 links proceed
4000 links proceed
4100 links proceed
4200 links proceed
4300 links proceed
4400 links proceed
4500 links proceed
4600 links proceed
4700 links proceed
4800 links proceed
4900 links proceed
5000 links proceed
5100 links proceed
5200 links proceed
5300 links proceed
54

In [34]:
columns_list =  ['adress', 'building_year', 'number_of_floors', 'type_house', 'series_of_house', 
                 'type_flooring', 'carry_wall_mater', 'is_garbage_chute', 'is_house_emergency', 
                 'year_of_population', 'is_child_area', 'is_sport_area', 'year_commissioning', 
                 'number_living_rooms', 'energy_safety_class', 'number_of_lifts', 
                 'number_of_entrances', 'lifts_types', 'max_num_floors', 'min_num_floors', 
                 'number_non_living_rooms', 'area_living_rooms', 'area_non_living_rooms', 
                 'area_collective_rooms', 'area_house_land', 'area_parking']
df_3 = pd.read_csv(DIRECTORY+PATH_3+'houses_3.csv', delimiter=';', names=columns_list)
df_3.head()

,adress,building_year,number_of_floors,type_house,series_of_house,type_flooring,carry_wall_mater,is_garbage_chute,is_house_emergency,year_of_population,...,number_of_entrances,lifts_types,max_num_floors,min_num_floors,number_non_living_rooms,area_living_rooms,area_non_living_rooms,area_collective_rooms,area_house_land,area_parking
0,"ш. 1-е мочищенское, д. 150, новосибирск, новос...",2012,9,многоквартирный дом,отсутствует,отсутствует,кирпич,отсутствует,нет,2012,...,9,не известно,9,не известно,не известно,9 593.00,7 977.00,не известно,не известно,не известно
1,"пер. краснодонский 1-й, д. 3, новосибирск, нов...",1957,5,многоквартирный дом,информация отсуствует,деревянные,кирпич,отсутствует,нет,1957,...,3,не известно,5,4,2,2 980.60,180.60,1 718.11,2 874.40,0.00
2,"пер. краснодонский 1-й, д. 4, новосибирск, нов...",1952,3,многоквартирный дом,информация отсутствует,деревянные,кирпич,отсутствует,нет,1952,...,3,не известно,3,3,не известно,1 856.70,0.00,1 548.30,2 521.00,0.00
3,"пер. краснодонский 1-й, д. 9, новосибирск, нов...",1954,4,многоквартирный дом,информация отсуствует,деревянные,кирпич,отсутствует,нет,1954,...,6,не известно,4,4,не известно,5 096.50,0.00,3 528.88,3 867.20,не известно
4,"пер. краснодонский 1-й, д. 10, новосибирск, но...",1953,4,многоквартирный дом,информация отсуствует,железобетонные,кирпич,отсутствует,нет,1953,...,2,не известно,4,4,не известно,1 840.90,0.00,1 073.52,2 649.00,0.00


In [37]:
v_cnts_3 = df_3.series_of_house.value_counts()
v_cnts_3.loc[v_cnts_3 > 50]

отсутствует                        1822
кирпичный                           948
-                                   599
панельный                           381
1-447                               377
нет                                 313
1-46                                226
типовая                             224
1-468                               189
1-464                               184
жилой дом                           183
блочный                             169
кирпич                              128
1-90                                112
шлакоблочный                        102
нет данных                           97
информация отсутствует               97
нет по тех. паспорту                 87
многоквартирный дом                  66
деревянный                           62
индивидуальный проект                59
97 серия, индивидуальный проект      58
90                                   55
индивидуальный                       53
1-464д                               51


In [38]:
# приближенная оценка числа данных по сериям домов, которым данные отсуствовали
(v_cnts_3.loc[v_cnts_3 > 50].sum()-377-226-189-184-112-58-55-51)/len(df_3)

0.6310736447722749